In [7]:
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
import scipy as sp
from scipy import signal
from statsmodels import robust
import glob
import io
import csv
import os
import pandas as pd
import json
import numpy as np
import random
gesture = ["spaghetti"]
path = "trimmed_data_updated/"+gesture[0]+"/"
data = {}
w = "20"
filenames = {"gyro"+ w +"-left2a", "gyro"+ w +"-right2a", "acc"+ w +"-left2a", "acc"+ w +"-right2a", "emg"+ w +"-left2a", "emg"+ w +"-right2a"};

for f in filenames:
    
    data[f] = pd.read_csv(path+f+".csv")


In [8]:
dataFrameDictionary = {}

arms = ["left", "right"]

emgDataFrames = {}
emgDataFrames['right'] = data["emg"+ w +"-right2a"]
emgDataFrames['left'] = data["emg"+ w +"-left2a"]

accDataFrames = {}
accDataFrames['right'] = data["acc"+ w +"-right2a"]
accDataFrames['left'] = data["acc"+ w +"-left2a"]

gyroDataFrames = {}
gyroDataFrames['right'] = data["gyro"+ w +"-right2a"]
gyroDataFrames['left'] = data["gyro"+ w +"-left2a"]

dataFrameDictionary['emg'] = emgDataFrames
dataFrameDictionary['acc'] = accDataFrames
dataFrameDictionary['gyro'] = gyroDataFrames

features ={}
features['right'] = []
features['left'] = []

for a in arms:
    
    df = dataFrameDictionary['emg'][a]
    
    # bandpass filter
    for column in df:
        if "time" not in column:
            emg = np.array(df[column].values)

            emg_correctmean = emg - np.mean(emg)

            # create bandpass filter for EMG
            high = 2/(100/2)
            low = 45/(100/2)
            b, a1 = sp.signal.butter(4, [high,low], btype='bandpass')

            emg_filtered = sp.signal.filtfilt(b, a1, emg_correctmean)

            emg_rectified = abs(emg_filtered)

                # create lowpass filter and apply to rectified signal to get EMG envelope
            sfreq = 100
            low_pass = 10/sfreq
            b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
            emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)

            df[column] = emg_envelope
    
    filteredDf = df
    
    print("Filtering of " + a + " emg Done!!")
    
    ctr = 0
   
    x = filteredDf.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
    normalizedDf = df_normalized

    print(a + "EMG Data Normalization done!!")

    

    ctr = 0
    
    df_normalized = normalizedDf
    
    arm = a + " "
    
    varianceDataFrame = pd.DataFrame(columns = [(arm + 'variance of EMG1'), (arm + 'variance of EMG2'), (arm + 'variance of EMG3'), (arm + 'variance of EMG4'),
                                 (arm + 'variance of EMG5'), (arm + 'variance of EMG6'), (arm + 'variance of EMG7'), (arm + 'variance of EMG8')])
    mavDataFrame = pd.DataFrame(columns = [(arm + 'Mav of EMG1'), (arm + 'Mav of EMG2'), (arm + 'Mav of EMG3'), (arm + 'Mav of EMG4'),
                                     (arm + 'Mav of EMG5'), (arm + 'Mav of EMG6'), (arm + 'Mav of EMG7'), (arm + 'Mav of EMG8')])
    meanDataFrame = pd.DataFrame(columns = [(arm + 'Mean of EMG1'), (arm + 'Mean of EMG2'), (arm + 'Mean of EMG3'), (arm + 'Mean of EMG4'),
                                     (arm + 'Mean of EMG5'), (arm + 'Mean of EMG6'), (arm + 'Mean of EMG7'), (arm + 'Mean of EMG8')])
    sdDataFrame = pd.DataFrame(columns = [(arm + 'Sd of EMG1'), (arm + 'Sd of EMG2'), (arm + 'Sd of EMG3'), (arm + 'Sd of EMG4'),
                                     (arm + 'Sd of EMG5'), (arm + 'Sd of EMG6'), (arm + 'Sd of EMG7'), (arm + 'Sd of EMG8')])
    rmsDataFrame = pd.DataFrame(columns = [(arm + 'Rms of EMG1'), (arm + 'Rms of EMG2'), (arm + 'Rms of EMG3'), (arm + 'Rms of EMG4'),
                                     (arm + 'Rms of EMG5'), (arm + 'Rms of EMG6'), (arm + 'Rms of EMG7'), (arm + 'Rms of EMG8')])
    meanFreqDataFrame = pd.DataFrame(columns = [(arm + 'MeanFreq of EMG1'), (arm + 'MeanFreq of EMG2'), (arm + 'MeanFreq of EMG3'), (arm + 'MeanFreq of EMG4'),
                                 (arm + 'MeanFreq of EMG5'), (arm + 'MeanFreq of EMG6'), (arm + 'MeanFreq of EMG7'), (arm + 'MeanFreq of EMG8')]) 
    medFreqDataFrame = pd.DataFrame(columns = [(arm + 'MedianFreq of EMG1'), (arm + 'MedianFreq of EMG2'), (arm + 'MedianFreq of EMG3'), (arm + 'MedianFreq of EMG4'),
                                     (arm + 'MedianFreq of EMG5'), (arm + 'MedianFreq of EMG6'), (arm + 'MedianFreq of EMG7'), (arm + 'MedianFreq of EMG8')])

    varianceDataFrame.loc[ctr] = [df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3], df_normalized.var()[4], df_normalized.var()[5], df_normalized.var()[6], df_normalized.var()[7], df_normalized.var()[8]]
    mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3], robust.mad(df_normalized)[4], robust.mad(df_normalized)[5], robust.mad(df_normalized)[6], robust.mad(df_normalized)[7], robust.mad(df_normalized)[8]]
    meanDataFrame.loc[ctr] = [np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3], np.mean(df_normalized)[4], np.mean(df_normalized)[5], np.mean(df_normalized)[6], np.mean(df_normalized)[7], np.mean(df_normalized)[8]]
    sdDataFrame.loc[ctr] = [np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3], np.std(df_normalized)[4], np.std(df_normalized)[5], np.std(df_normalized)[6], np.std(df_normalized)[7], np.std(df_normalized)[8]]
    rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2)), np.sqrt(np.mean(df_normalized[4].values**2)), np.sqrt(np.mean(df_normalized[5].values**2)), np.sqrt(np.mean(df_normalized[6].values**2)),  np.sqrt(np.mean(df_normalized[7].values**2)), np.sqrt(np.mean(df_normalized[8].values**2))]
    
    meanFreq = []
    medFreq = []

    for i in range(1, 9):
        data = df_normalized[i].values
        ps = np.abs(np.fft.fft(data))**2

        freqs = np.fft.fftfreq(data.size, 1/100)

        sumProd = 0
        sumPs = 0

        for f, p in zip(freqs,ps):
            sumProd += f*p
            sumPs += p

        sumProd = sumProd * 100000000
        meanFreq.append(100000000*sumProd/sumPs)
        medFreq.append(sumPs/2)


    meanFreqDataFrame.loc[ctr] = [meanFreq[0], meanFreq[1], meanFreq[2], meanFreq[3], meanFreq[4], meanFreq[5], meanFreq[6], meanFreq[7]]
    medFreqDataFrame.loc[ctr] = [medFreq[0], medFreq[1], medFreq[2], medFreq[3], medFreq[4], medFreq[5], medFreq[6], medFreq[7]]

    ctr +=1 

    features[a].append(varianceDataFrame)
    features[a].append(mavDataFrame)
    features[a].append(meanDataFrame)
    features[a].append(sdDataFrame)
    features[a].append(rmsDataFrame)
    features[a].append(meanFreqDataFrame)
    features[a].append(medFreqDataFrame)
    print("Done computing emg features of " + str(ctr) + a + " arm files")


Filtering of left emg Done!!
leftEMG Data Normalization done!!
Done computing emg features of 1left arm files
Filtering of right emg Done!!
rightEMG Data Normalization done!!
Done computing emg features of 1right arm files


In [9]:
for a in arms:
    df = dataFrameDictionary['acc'][a]

    x = df.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
    normalizedAccDf = df_normalized

    print(a + "Acc Data Normalization done!!")

    ctr = 0    
    arm = a + " "
    
    
    #Acc features
    df_normalized = normalizedAccDf
    
    varianceDataFrame = pd.DataFrame(columns = [(arm+ 'Variance of ACC x'), (arm+ 'Variance of ACC y'), (arm+ 'Variance of ACC z')])
    mavDataFrame = pd.DataFrame(columns = [(arm + 'Mav of ACC x'), (arm + 'Mav of ACC y'), (arm + 'Mav of ACC z')])
    meanDataFrame = pd.DataFrame(columns = [(arm + 'Mean of ACC x'), (arm + 'Mean of ACC y'), (arm + 'Mean of ACC z')])
    sdDataFrame = pd.DataFrame(columns = [(arm + 'Sd of ACC x'), (arm + 'Sd of ACC y'), (arm + 'Sd of ACC z')])
    rmsDataFrame = pd.DataFrame(columns = [(arm + 'Rms of ACC x'), (arm + 'Rms of ACC y'), (arm + 'Rms of ACC z')])
    entropyDataFrame = pd.DataFrame(columns = [(arm + 'Entropy of ACC x'), (arm + 'Entropy of ACC y'), (arm + 'Entropy of ACC z')])
    kurtosisDataFrame = pd.DataFrame(columns = [(arm + 'Kurtosis of ACC x'), (arm + 'Kurtosis of ACC y'), (arm + 'Kurtosis of ACC z')])
    skewnessDataFrame = pd.DataFrame(columns = [(arm + 'Skewness of ACC x'), (arm + 'Skewness of ACC y'), (arm + 'Skewness of ACC z')])
    smaDataFrame = pd.DataFrame(columns = [(arm + 'SMA of ACC')])
    integrationDataFrame = pd.DataFrame(columns = [(arm + 'Integration of ACC x'), (arm + 'Integration of ACC y'), (arm + 'Integration of ACC z')])
    
    
    
    varianceDataFrame.loc[ctr] = [df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3]]
    mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3]]
    meanDataFrame.loc[ctr] = [np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3]]
    sdDataFrame.loc[ctr] = [np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3]]
    rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2))]
    entropyDataFrame.loc[ctr] = [sp.stats.entropy(df_normalized)[1], sp.stats.entropy(df_normalized)[2], sp.stats.entropy(df_normalized)[3]]
    kurtosisDataFrame.loc[ctr] = [df_normalized.kurtosis()[1], df_normalized.kurtosis()[2], df_normalized.kurtosis()[3]]
    skewnessDataFrame.loc[ctr] = [df_normalized.skew(axis=0)[1], df_normalized.skew(axis=0)[2], df_normalized.skew(axis=0)[3]]
    length = df_normalized.shape[1]
    sma = 0
    for x in range(0, length):
        sma += abs(df_normalized.iloc[x,1]) + df_normalized.iloc[x,2] + df_normalized.iloc[x,3]
    sma = sma / length
    smaDataFrame.loc[ctr] = [sma]
    integrationDataFrame.loc[ctr] = [sp.integrate.simps(df_normalized, axis=0)[1],sp.integrate.simps(df_normalized, axis=0)[2], sp.integrate.simps(df_normalized, axis=0)[3]]

    features[a].append(varianceDataFrame)
    features[a].append(mavDataFrame)
    features[a].append(meanDataFrame)
    features[a].append(sdDataFrame)
    features[a].append(rmsDataFrame)
    features[a].append(entropyDataFrame) 
    features[a].append(kurtosisDataFrame) 
    features[a].append(skewnessDataFrame) 
    features[a].append(smaDataFrame) 
    features[a].append(integrationDataFrame) 
    
    print(a +"Done computing acc features!!!!!!!")
    
    df = dataFrameDictionary['gyro'][a]
    x = df.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
    normalizedGyroDf = df_normalized
    print(a +"Gyro Data Normalization done!!")
    
    #Gyro features    
    df_normalized = normalizedGyroDf
    varianceDataFrame = pd.DataFrame(columns = [(arm + 'Variance of Gyro x'), (arm+ 'Variance of Gyro y'), (arm+ 'Variance of Gyro z')])
    mavDataFrame = pd.DataFrame(columns = [(arm+ 'Mav of Gyro x'), (arm+ 'Mav of Gyro y'), (arm+ 'Mav of Gyro z')])
    meanDataFrame = pd.DataFrame(columns = [(arm+ 'Mean of Gyro x'), (arm+ 'Mean of Gyro y'), (arm+ 'Mean of Gyro z')])
    sdDataFrame = pd.DataFrame(columns = [(arm+ 'Sd of Gyro x'), (arm+ 'Sd of Gyro y'), (arm+ 'Sd of Gyro z')])
    rmsDataFrame = pd.DataFrame(columns = [(arm+ 'Rms of Gyro x'), (arm+ 'Rms of Gyro y'), (arm+ 'Rms of Gyro z')])
    entropyDataFrame = pd.DataFrame(columns = [(arm+ 'Entropy of Gyro x'), (arm+ 'Entropy of Gyro y'), (arm+ 'Entropy of Gyro z')])
    kurtosisDataFrame = pd.DataFrame(columns = [(arm+ 'Kurtosis of Gyro x'), (arm+ 'Kurtosis of Gyro y'), (arm+ 'Kurtosis of Gyro z')])
    skewnessDataFrame = pd.DataFrame(columns = [(arm+ 'Skewness of Gyro x'), (arm+ 'Skewness of Gyro y'), (arm+ 'Skewness of Gyro z')])
    smaDataFrame = pd.DataFrame(columns = [(arm+ 'SMA of Gyro')])
    integrationDataFrame = pd.DataFrame(columns = [(arm+ 'Integration of Gyro x'), (arm+ 'Integration of Gyro y'), (arm+ 'Integration of Gyro z')])

        
    varianceDataFrame.loc[ctr] = [df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3]]
    mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3]]
    meanDataFrame.loc[ctr] = [np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3]]
    sdDataFrame.loc[ctr] = [np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3]]
    rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2))]
    entropyDataFrame.loc[ctr] = [sp.stats.entropy(df_normalized)[1], sp.stats.entropy(df_normalized)[2], sp.stats.entropy(df_normalized)[3],]
    kurtosisDataFrame.loc[ctr] = [df_normalized.kurtosis()[1], df_normalized.kurtosis()[2], df_normalized.kurtosis()[3]]
    skewnessDataFrame.loc[ctr] = [df_normalized.skew(axis=0)[1], df_normalized.skew(axis=0)[2], df_normalized.skew(axis=0)[3]]
    length = df_normalized.shape[1]
    sma = 0
    for x in range(0, length):
            # .iloc[x,0] is Gyroessing row, column
        sma += abs(df_normalized.iloc[x,1]) + df_normalized.iloc[x,2] + df_normalized.iloc[x,3]
    sma = sma / length
    smaDataFrame.loc[ctr] = [sma]
    integrationDataFrame.loc[ctr] = [sp.integrate.simps(df_normalized, axis=0)[1],sp.integrate.simps(df_normalized, axis=0)[2], sp.integrate.simps(df_normalized, axis=0)[3]]

    features[a].append(varianceDataFrame)
    features[a].append(mavDataFrame)
    features[a].append(meanDataFrame)
    features[a].append(sdDataFrame)
    features[a].append(rmsDataFrame)
    features[a].append(entropyDataFrame) 
    features[a].append(kurtosisDataFrame) 
    features[a].append(skewnessDataFrame) 
    features[a].append(smaDataFrame) 
    features[a].append(integrationDataFrame) 
    
    print(a +"Done computing gyro features!!!!!!!")


leftAcc Data Normalization done!!
leftDone computing acc features!!!!!!!
leftGyro Data Normalization done!!
leftDone computing gyro features!!!!!!!
rightAcc Data Normalization done!!
rightDone computing acc features!!!!!!!
rightGyro Data Normalization done!!
rightDone computing gyro features!!!!!!!


In [10]:
#combine data
firstFeature = features['left'][0]
consolidatedFeatures = firstFeature

for a in arms:
    for feature in features[a]:
        for column in feature:
            if('left variance of G' not in column):
                consolidatedFeatures[column] = feature[column]
                


In [11]:
fileLocation = "WithEmgFilter-NoNormalization.csv"
df = pd.read_csv(fileLocation)

for f in df:
   
    if(f != "Class"):
        mini = min(df[f])
        maxi = max(df[f])
        norm = (consolidatedFeatures[f].iloc[0]-mini) / (maxi - mini) 
        consolidatedFeatures.set_value(0, f, norm)
    
print(consolidatedFeatures)

   left variance of EMG1  left variance of EMG2  left variance of EMG3  \
0               0.140983               0.504153               0.358176   

   left variance of EMG4  left variance of EMG5  left variance of EMG6  \
0               0.264634               0.105019               0.481689   

   left variance of EMG7  left variance of EMG8  left Mav of EMG1  \
0               0.326091               0.528146           0.14956   

   left Mav of EMG2             ...               right Kurtosis of Gyro x  \
0          0.477464             ...                                0.07317   

   right Kurtosis of Gyro y  right Kurtosis of Gyro z  \
0                  0.131564                  0.008102   

   right Skewness of Gyro x  right Skewness of Gyro y  \
0                  0.450144                  0.454977   

   right Skewness of Gyro z  right SMA of Gyro  right Integration of Gyro x  \
0                  0.389794           0.497005                     0.362477   

   right Integrat

In [12]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

fileLocation = "WithEmgFilter.csv"
df = pd.read_csv(fileLocation)

data = df.drop('Class', axis=1)  
classes = df['Class']

X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size = 0.20, random_state=0)


clf = SVC(kernel='linear')
clf.fit(data, classes)  

y_pred = clf.predict(consolidatedFeatures)  

print(y_pred)
print(confusion_matrix(gesture,y_pred))  
print(classification_report(gesture,y_pred))  
print(accuracy_score(gesture, y_pred))

['t-shirt']
[[0 1]
 [0 0]]
             precision    recall  f1-score   support

  spaghetti       0.00      0.00      0.00         1
    t-shirt       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00         1

0.0


C:\Users\darre\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\darre\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
import scipy as sp
from scipy import signal
from statsmodels import robust
import glob
import io
import csv
import os
import pandas as pd
import json
import numpy as np
import random
gesture = ["spaghetti"]
path = "trimmed_data_updated/"+gesture[0]+"/"

for ir in range(0, 100):
    data = {}
    w = str(ir)
    filenames = {"gyro"+ w +"-left2a", "gyro"+ w +"-right2a", "acc"+ w +"-left2a", "acc"+ w +"-right2a", "emg"+ w +"-left2a", "emg"+ w +"-right2a"};

    for f in filenames:

        data[f] = pd.read_csv(path+f+".csv")
    dataFrameDictionary = {}

    arms = ["left", "right"]

    emgDataFrames = {}
    emgDataFrames['right'] = data["emg"+ w +"-right2a"]
    emgDataFrames['left'] = data["emg"+ w +"-left2a"]

    accDataFrames = {}
    accDataFrames['right'] = data["acc"+ w +"-right2a"]
    accDataFrames['left'] = data["acc"+ w +"-left2a"]

    gyroDataFrames = {}
    gyroDataFrames['right'] = data["gyro"+ w +"-right2a"]
    gyroDataFrames['left'] = data["gyro"+ w +"-left2a"]

    dataFrameDictionary['emg'] = emgDataFrames
    dataFrameDictionary['acc'] = accDataFrames
    dataFrameDictionary['gyro'] = gyroDataFrames

    features ={}
    features['right'] = []
    features['left'] = []

    for a in arms:

        df = dataFrameDictionary['emg'][a]

        # bandpass filter
        for column in df:
            if "time" not in column:
                emg = np.array(df[column].values)

                emg_correctmean = emg - np.mean(emg)

                # create bandpass filter for EMG
                high = 2/(100/2)
                low = 45/(100/2)
                b, a1 = sp.signal.butter(4, [high,low], btype='bandpass')

                emg_filtered = sp.signal.filtfilt(b, a1, emg_correctmean)

                emg_rectified = abs(emg_filtered)

                    # create lowpass filter and apply to rectified signal to get EMG envelope
                sfreq = 100
                low_pass = 10/sfreq
                b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
                emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)

                df[column] = emg_envelope

        filteredDf = df


        ctr = 0

        x = filteredDf.values.astype(float)
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled)
        normalizedDf = df_normalized

      


        ctr = 0

        df_normalized = normalizedDf

        arm = a + " "

        varianceDataFrame = pd.DataFrame(columns = [(arm + 'variance of EMG1'), (arm + 'variance of EMG2'), (arm + 'variance of EMG3'), (arm + 'variance of EMG4'),
                                     (arm + 'variance of EMG5'), (arm + 'variance of EMG6'), (arm + 'variance of EMG7'), (arm + 'variance of EMG8')])
        mavDataFrame = pd.DataFrame(columns = [(arm + 'Mav of EMG1'), (arm + 'Mav of EMG2'), (arm + 'Mav of EMG3'), (arm + 'Mav of EMG4'),
                                         (arm + 'Mav of EMG5'), (arm + 'Mav of EMG6'), (arm + 'Mav of EMG7'), (arm + 'Mav of EMG8')])
        meanDataFrame = pd.DataFrame(columns = [(arm + 'Mean of EMG1'), (arm + 'Mean of EMG2'), (arm + 'Mean of EMG3'), (arm + 'Mean of EMG4'),
                                         (arm + 'Mean of EMG5'), (arm + 'Mean of EMG6'), (arm + 'Mean of EMG7'), (arm + 'Mean of EMG8')])
        sdDataFrame = pd.DataFrame(columns = [(arm + 'Sd of EMG1'), (arm + 'Sd of EMG2'), (arm + 'Sd of EMG3'), (arm + 'Sd of EMG4'),
                                         (arm + 'Sd of EMG5'), (arm + 'Sd of EMG6'), (arm + 'Sd of EMG7'), (arm + 'Sd of EMG8')])
        rmsDataFrame = pd.DataFrame(columns = [(arm + 'Rms of EMG1'), (arm + 'Rms of EMG2'), (arm + 'Rms of EMG3'), (arm + 'Rms of EMG4'),
                                         (arm + 'Rms of EMG5'), (arm + 'Rms of EMG6'), (arm + 'Rms of EMG7'), (arm + 'Rms of EMG8')])
        meanFreqDataFrame = pd.DataFrame(columns = [(arm + 'MeanFreq of EMG1'), (arm + 'MeanFreq of EMG2'), (arm + 'MeanFreq of EMG3'), (arm + 'MeanFreq of EMG4'),
                                     (arm + 'MeanFreq of EMG5'), (arm + 'MeanFreq of EMG6'), (arm + 'MeanFreq of EMG7'), (arm + 'MeanFreq of EMG8')]) 
        medFreqDataFrame = pd.DataFrame(columns = [(arm + 'MedianFreq of EMG1'), (arm + 'MedianFreq of EMG2'), (arm + 'MedianFreq of EMG3'), (arm + 'MedianFreq of EMG4'),
                                         (arm + 'MedianFreq of EMG5'), (arm + 'MedianFreq of EMG6'), (arm + 'MedianFreq of EMG7'), (arm + 'MedianFreq of EMG8')])

        varianceDataFrame.loc[ctr] = [df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3], df_normalized.var()[4], df_normalized.var()[5], df_normalized.var()[6], df_normalized.var()[7], df_normalized.var()[8]]
        mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3], robust.mad(df_normalized)[4], robust.mad(df_normalized)[5], robust.mad(df_normalized)[6], robust.mad(df_normalized)[7], robust.mad(df_normalized)[8]]
        meanDataFrame.loc[ctr] = [np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3], np.mean(df_normalized)[4], np.mean(df_normalized)[5], np.mean(df_normalized)[6], np.mean(df_normalized)[7], np.mean(df_normalized)[8]]
        sdDataFrame.loc[ctr] = [np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3], np.std(df_normalized)[4], np.std(df_normalized)[5], np.std(df_normalized)[6], np.std(df_normalized)[7], np.std(df_normalized)[8]]
        rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2)), np.sqrt(np.mean(df_normalized[4].values**2)), np.sqrt(np.mean(df_normalized[5].values**2)), np.sqrt(np.mean(df_normalized[6].values**2)),  np.sqrt(np.mean(df_normalized[7].values**2)), np.sqrt(np.mean(df_normalized[8].values**2))]

        meanFreq = []
        medFreq = []

        for i in range(1, 9):
            data = df_normalized[i].values
            ps = np.abs(np.fft.fft(data))**2

            freqs = np.fft.fftfreq(data.size, 1/100)

            sumProd = 0
            sumPs = 0

            for f, p in zip(freqs,ps):
                sumProd += f*p
                sumPs += p

            sumProd = sumProd * 100000000
            meanFreq.append(100000000*sumProd/sumPs)
            medFreq.append(sumPs/2)


        meanFreqDataFrame.loc[ctr] = [meanFreq[0], meanFreq[1], meanFreq[2], meanFreq[3], meanFreq[4], meanFreq[5], meanFreq[6], meanFreq[7]]
        medFreqDataFrame.loc[ctr] = [medFreq[0], medFreq[1], medFreq[2], medFreq[3], medFreq[4], medFreq[5], medFreq[6], medFreq[7]]

        ctr +=1 

        features[a].append(varianceDataFrame)
        features[a].append(mavDataFrame)
        features[a].append(meanDataFrame)
        features[a].append(sdDataFrame)
        features[a].append(rmsDataFrame)
        features[a].append(meanFreqDataFrame)
        features[a].append(medFreqDataFrame)
      
    for a in arms:
        df = dataFrameDictionary['acc'][a]

        x = df.values.astype(float)
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled)
        normalizedAccDf = df_normalized

      
        ctr = 0    
        arm = a + " "


        #Acc features
        df_normalized = normalizedAccDf

        varianceDataFrame = pd.DataFrame(columns = [(arm+ 'Variance of ACC x'), (arm+ 'Variance of ACC y'), (arm+ 'Variance of ACC z')])
        mavDataFrame = pd.DataFrame(columns = [(arm + 'Mav of ACC x'), (arm + 'Mav of ACC y'), (arm + 'Mav of ACC z')])
        meanDataFrame = pd.DataFrame(columns = [(arm + 'Mean of ACC x'), (arm + 'Mean of ACC y'), (arm + 'Mean of ACC z')])
        sdDataFrame = pd.DataFrame(columns = [(arm + 'Sd of ACC x'), (arm + 'Sd of ACC y'), (arm + 'Sd of ACC z')])
        rmsDataFrame = pd.DataFrame(columns = [(arm + 'Rms of ACC x'), (arm + 'Rms of ACC y'), (arm + 'Rms of ACC z')])
        entropyDataFrame = pd.DataFrame(columns = [(arm + 'Entropy of ACC x'), (arm + 'Entropy of ACC y'), (arm + 'Entropy of ACC z')])
        kurtosisDataFrame = pd.DataFrame(columns = [(arm + 'Kurtosis of ACC x'), (arm + 'Kurtosis of ACC y'), (arm + 'Kurtosis of ACC z')])
        skewnessDataFrame = pd.DataFrame(columns = [(arm + 'Skewness of ACC x'), (arm + 'Skewness of ACC y'), (arm + 'Skewness of ACC z')])
        smaDataFrame = pd.DataFrame(columns = [(arm + 'SMA of ACC')])
        integrationDataFrame = pd.DataFrame(columns = [(arm + 'Integration of ACC x'), (arm + 'Integration of ACC y'), (arm + 'Integration of ACC z')])



        varianceDataFrame.loc[ctr] = [df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3]]
        mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3]]
        meanDataFrame.loc[ctr] = [np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3]]
        sdDataFrame.loc[ctr] = [np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3]]
        rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2))]
        entropyDataFrame.loc[ctr] = [sp.stats.entropy(df_normalized)[1], sp.stats.entropy(df_normalized)[2], sp.stats.entropy(df_normalized)[3]]
        kurtosisDataFrame.loc[ctr] = [df_normalized.kurtosis()[1], df_normalized.kurtosis()[2], df_normalized.kurtosis()[3]]
        skewnessDataFrame.loc[ctr] = [df_normalized.skew(axis=0)[1], df_normalized.skew(axis=0)[2], df_normalized.skew(axis=0)[3]]
        length = df_normalized.shape[1]
        sma = 0
        for x in range(0, length):
            sma += abs(df_normalized.iloc[x,1]) + df_normalized.iloc[x,2] + df_normalized.iloc[x,3]
        sma = sma / length
        smaDataFrame.loc[ctr] = [sma]
        integrationDataFrame.loc[ctr] = [sp.integrate.simps(df_normalized, axis=0)[1],sp.integrate.simps(df_normalized, axis=0)[2], sp.integrate.simps(df_normalized, axis=0)[3]]

        features[a].append(varianceDataFrame)
        features[a].append(mavDataFrame)
        features[a].append(meanDataFrame)
        features[a].append(sdDataFrame)
        features[a].append(rmsDataFrame)
        features[a].append(entropyDataFrame) 
        features[a].append(kurtosisDataFrame) 
        features[a].append(skewnessDataFrame) 
        features[a].append(smaDataFrame) 
        features[a].append(integrationDataFrame) 

      

        df = dataFrameDictionary['gyro'][a]
        x = df.values.astype(float)
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled)
        normalizedGyroDf = df_normalized
      

        #Gyro features    
        df_normalized = normalizedGyroDf
        varianceDataFrame = pd.DataFrame(columns = [(arm + 'Variance of Gyro x'), (arm+ 'Variance of Gyro y'), (arm+ 'Variance of Gyro z')])
        mavDataFrame = pd.DataFrame(columns = [(arm+ 'Mav of Gyro x'), (arm+ 'Mav of Gyro y'), (arm+ 'Mav of Gyro z')])
        meanDataFrame = pd.DataFrame(columns = [(arm+ 'Mean of Gyro x'), (arm+ 'Mean of Gyro y'), (arm+ 'Mean of Gyro z')])
        sdDataFrame = pd.DataFrame(columns = [(arm+ 'Sd of Gyro x'), (arm+ 'Sd of Gyro y'), (arm+ 'Sd of Gyro z')])
        rmsDataFrame = pd.DataFrame(columns = [(arm+ 'Rms of Gyro x'), (arm+ 'Rms of Gyro y'), (arm+ 'Rms of Gyro z')])
        entropyDataFrame = pd.DataFrame(columns = [(arm+ 'Entropy of Gyro x'), (arm+ 'Entropy of Gyro y'), (arm+ 'Entropy of Gyro z')])
        kurtosisDataFrame = pd.DataFrame(columns = [(arm+ 'Kurtosis of Gyro x'), (arm+ 'Kurtosis of Gyro y'), (arm+ 'Kurtosis of Gyro z')])
        skewnessDataFrame = pd.DataFrame(columns = [(arm+ 'Skewness of Gyro x'), (arm+ 'Skewness of Gyro y'), (arm+ 'Skewness of Gyro z')])
        smaDataFrame = pd.DataFrame(columns = [(arm+ 'SMA of Gyro')])
        integrationDataFrame = pd.DataFrame(columns = [(arm+ 'Integration of Gyro x'), (arm+ 'Integration of Gyro y'), (arm+ 'Integration of Gyro z')])


        varianceDataFrame.loc[ctr] = [df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3]]
        mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3]]
        meanDataFrame.loc[ctr] = [np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3]]
        sdDataFrame.loc[ctr] = [np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3]]
        rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2))]
        entropyDataFrame.loc[ctr] = [sp.stats.entropy(df_normalized)[1], sp.stats.entropy(df_normalized)[2], sp.stats.entropy(df_normalized)[3],]
        kurtosisDataFrame.loc[ctr] = [df_normalized.kurtosis()[1], df_normalized.kurtosis()[2], df_normalized.kurtosis()[3]]
        skewnessDataFrame.loc[ctr] = [df_normalized.skew(axis=0)[1], df_normalized.skew(axis=0)[2], df_normalized.skew(axis=0)[3]]
        length = df_normalized.shape[1]
        sma = 0
        for x in range(0, length):
                # .iloc[x,0] is Gyroessing row, column
            sma += abs(df_normalized.iloc[x,1]) + df_normalized.iloc[x,2] + df_normalized.iloc[x,3]
        sma = sma / length
        smaDataFrame.loc[ctr] = [sma]
        integrationDataFrame.loc[ctr] = [sp.integrate.simps(df_normalized, axis=0)[1],sp.integrate.simps(df_normalized, axis=0)[2], sp.integrate.simps(df_normalized, axis=0)[3]]

        features[a].append(varianceDataFrame)
        features[a].append(mavDataFrame)
        features[a].append(meanDataFrame)
        features[a].append(sdDataFrame)
        features[a].append(rmsDataFrame)
        features[a].append(entropyDataFrame) 
        features[a].append(kurtosisDataFrame) 
        features[a].append(skewnessDataFrame) 
        features[a].append(smaDataFrame) 
        features[a].append(integrationDataFrame) 

      

    #combine data
    firstFeature = features['left'][0]
    consolidatedFeatures = firstFeature

    for a in arms:
        for feature in features[a]:
            for column in feature:
                if('left variance of E' not in column):
                    consolidatedFeatures[column] = feature[column]

    fileLocation = "WithEmgFilter-NoNormalization.csv"
    df = pd.read_csv(fileLocation)

    for f in df:

        if(f != "Class"):
            mini = min(df[f])
            maxi = max(df[f])
            norm = (consolidatedFeatures[f].iloc[0]-mini) / (maxi - mini) 
            consolidatedFeatures.set_value(0, f, norm)


    import pandas as pd
    import numpy as np
    import random
    import csv
    from sklearn.svm import SVC, LinearSVC
    from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

    fileLocation = "WithEmgFilter.csv"
    df = pd.read_csv(fileLocation)

    data = df.drop('Class', axis=1)  
    classes = df['Class']

    X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size = 0.20, random_state=0)


    clf = SVC(kernel='linear')
    clf.fit(data, classes)  

    y_pred = clf.predict(consolidatedFeatures)  

    print(y_pred)
    print(y_pred[0] == gesture[0])  